In [38]:
from PIL import Image
import requests
import numpy as np
from PIL import Image

import pandas as pd 
import geopandas as gpd
# import osmnx as ox
# import networkx as nx

import pyproj
from shapely.ops import transform

import shapely
from shapely import Point

# import boto3
import os
import django

pd.set_option('display.precision', 2)

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "water.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from water.models import ReservoirState, Reservoir, ReservoirStateSerializer, RainFall

from water.utils import parse as p
from django.db.models import Count

In [59]:
rf = ReservoirState.objects.all()[0]
vals = {}
vals['date'] = rf.date
vals['reservoir_state'] = rf

from rest_framework import serializers
class DailyDataSerializer(serializers.Serializer):
    date = serializers.DateField()
    reservoir_state = ReservoirStateSerializer()

DailyDataSerializer(vals).data

{'date': '2012-09-01', 'reservoir_state': {'uuid': 'b6cdceb0-147a-420c-a601-cf9d3c778ef7', 'date': '2012-09-01', 'volume': 30.67, 'reservoir': {'uuid': '20bb971a-4c4a-43dd-aa92-4f72c20ba351', 'name': 'conde de guadalhorce', 'capacity': 66.5}}}

In [50]:
RainFall.objects.all()

<QuerySet []>

In [46]:
ReservoirState.objects.all()[0].reservoir.name

'conde de guadalhorce'

In [33]:
reservoirs = Reservoir.objects.annotate(
        num_states=Count("reservoir_reservoirstate")
    )

# Get all reservoirs with num_states > 0
reservoirs = reservoirs.filter(num_states__gt=0)
reservoirs[0].num_states


100

In [36]:
reservoirs[0].name

'conde de guadalhorce'